In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,preprocessing
from sklearn.compose import ColumnTransformer

from matplotlib.colors import ListedColormap

#классификация
from sklearn.linear_model import LogisticRegression #логистическая регрессия
from sklearn.neural_network import MLPClassifier #нейронка
from sklearn.naive_bayes import GaussianNB #наивный байес
from sklearn.neighbors import KNeighborsClassifier # k ближайших соседей

#регрессия
from sklearn.linear_model import LinearRegression #линейная регрессия
from sklearn.svm import SVC #метод опорных векторов
from sklearn.metrics import mean_squared_error


#Оценка качества в задачах классификации
from sklearn.metrics import f1_score, silhouette_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.menrics import roc_curve
from sklearn.metrics import precision_recall_curve

#Оценка качества в задачах регрессии
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

from sklearn import preprocessing

ModuleNotFoundError: No module named 'pandas'

In [2]:
#Задача: вычисление значения параметра code на основании остальных параметров.
train_data = pd.read_csv("Набор_2_цены_квартир.csv",delimiter=';')#Меняем разделитель на ;


In [3]:
#выбираем рандомное число
my_code = "Магдиев"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [4]:
# Определим размер валидационной выборки
val_size = round(0.2*len(train_data))
print(val_size)

408


In [10]:
# Создадим обучающую и валидационную выборки
random_state = my_seed
train, val = train_test_split(train_data, 
                              test_size=val_size, 
                              random_state=random_state)
print(len(train), len(val))

1632 408


In [7]:
print(train)

      price  totsp  livesp  kitsp  dist  metrdist  walk  brick  floor  code
524      78     63      47    6.0  13.5         5     0      1      1     6
1275    120     74      53    8.0  10.5        10     1      1      1     7
801     125     80      45   13.0  12.0         5     0      1      1     5
48      102     60      43    6.0  12.0         5     0      1      1     7
1903    130     72      45   10.0  11.5        10     1      0      0     6
...     ...    ...     ...    ...   ...       ...   ...    ...    ...   ...
75      110     81      45   10.0  13.0        12     1      0      1     5
804     230    110      52   17.0  11.5         5     1      1      1     6
498     150    108      56   15.0  11.0         5     1      1      1     5
1689     82     56      42    6.0  10.0        10     1      0      0     6
1070    105     72      43   10.0  13.0        10     1      0      0     5

[1632 rows x 10 columns]


In [8]:
print(val)

      price  totsp  livesp  kitsp  dist  metrdist  walk  brick  floor  code
727     128     78      54    9.0   6.5         5     1      0      1     5
1157    120     75      48   11.0   3.0        15     1      1      1     1
1188    120     76      45   10.0  12.0        10     0      0      1     7
450     155     75      46   10.0  12.0         5     1      0      1     8
615      81     53      34    6.0  13.0        15     1      0      0     5
...     ...    ...     ...    ...   ...       ...   ...    ...    ...   ...
1815     85     65      46    7.0  13.0         7     0      0      1     5
1918    205     87      58    9.0   7.5         5     1      1      1     3
396      90     65      45    7.0  12.0         7     1      0      1     5
1068    120     64      43    6.0  11.5         2     1      1      1     6
26      135     82      46   13.0  12.0         5     1      0      1     5

[408 rows x 10 columns]


In [11]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['price','totsp','livesp','kitsp','dist','metrdist','walk','brick','floor']
ord_columns = ['code']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')

ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['price', 'totsp', 'livesp', 'kitsp', 'dist',
                                  'metrdist', 'walk', 'brick', 'floor'])])

In [13]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_val = pd.DataFrame(ct.transform(val))

# даем название столбцам
column_names = num_columns + ord_columns
sc_train.columns = column_names
sc_val.columns = column_names

sc_train

,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,code
0,0.041176,0.097902,0.214286,0.052632,0.750000,0.210526,0.0,1.0,1.0,6.0
1,0.102941,0.174825,0.300000,0.157895,0.535714,0.473684,1.0,1.0,1.0,7.0
2,0.110294,0.216783,0.185714,0.421053,0.642857,0.210526,0.0,1.0,1.0,5.0
3,0.076471,0.076923,0.157143,0.052632,0.642857,0.210526,0.0,1.0,1.0,7.0
4,0.117647,0.160839,0.185714,0.263158,0.607143,0.473684,1.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...
1627,0.088235,0.223776,0.185714,0.263158,0.714286,0.578947,1.0,0.0,1.0,5.0
1628,0.264706,0.426573,0.285714,0.631579,0.607143,0.210526,1.0,1.0,1.0,6.0
1629,0.147059,0.412587,0.342857,0.526316,0.571429,0.210526,1.0,1.0,1.0,5.0
1630,0.047059,0.048951,0.142857,0.052632,0.500000,0.473684,1.0,0.0,0.0,6.0


In [9]:
# В качестве входных параметров используем первые 9 числовых параметров,
# в качестве выходного - десятый числовой параметр.
x_train = sc_train[num_columns]
x_val = sc_val[num_columns]

y_train = (sc_train[ord_columns].values).flatten()
y_val = (sc_val[ord_columns].values).flatten()

In [10]:
y_train

array([6., 7., 5., ..., 5., 6., 5.])

In [11]:
#Создаем модели
models = [MLPClassifier(alpha=1, max_iter=1000),
              LogisticRegression()]

In [12]:
#Обучаем модели
for model in models:
    print(model)
    model.fit(x_train, y_train)

MLPClassifier(alpha=1, max_iter=1000)
LogisticRegression()


In [13]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(model, '\t', f1)

MLPClassifier(alpha=1, max_iter=1000) 	 0.21782867400561365
LogisticRegression() 	 0.2492135301271371


In [14]:
#выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = models[i_max]
best_c_model

LogisticRegression()

In [15]:
#Загружаем рандомные данные квартир в Москве
test1 = pd.read_csv("рандомные данные_Цены квартир в Москве.csv")

In [16]:
test1['code'] = ''

In [17]:
sc_test = pd.DataFrame(ct.transform(test1))
sc_test.columns = column_names

In [18]:
x_test = sc_test[num_columns]

In [19]:
#вставляем данные работы первой модели
test1['code'] = models[0].predict(x_test)

In [20]:
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_val, val_pred, average='weighted')
print(f1)

0.2492135301271371


In [21]:
#создаем готовый csv файл с работой первой модели
test1.to_csv('нейронная сеть.csv', index=False)
test1.head()

,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,code
0,508,166,82,5,15,11,0,0,0,3.0
1,540,130,86,19,11,15,1,1,1,3.0
2,656,160,74,8,15,2,0,0,0,3.0
3,98,176,49,24,3,6,1,1,1,4.0
4,330,93,41,11,15,14,1,0,0,2.0


In [22]:
#Загружаем рандомные данные квартир в Москве

test2 = pd.read_csv("рандомные данные_Цены квартир в Москве.csv")

In [23]:
test2['code'] = ''

In [24]:
sc_test = pd.DataFrame(ct.transform(test2))
sc_test.columns = column_names

In [25]:
x_test = sc_test[num_columns]

In [26]:
#вставляем данные работы второй модели
test2['code'] = models[1].predict(x_test)

In [27]:
#создаем готовый csv файл с работой первой модели
test2.to_csv('логистическая регрессия.csv', index=False)
test2.head()

,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,code
0,508,166,82,5,15,11,0,0,0,3.0
1,540,130,86,19,11,15,1,1,1,3.0
2,656,160,74,8,15,2,0,0,0,3.0
3,98,176,49,24,3,6,1,1,1,5.0
4,330,93,41,11,15,14,1,0,0,2.0
